In [ ]:
import io
import os
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("data_sheet.csv")

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Number,Caption,Like,URL,Label,Label_len,Binary_Engagement
0,0,0,1,Photo by @estherhorvath | Moss-covered trees a...,97727,https://scontent-hou1-1.cdninstagram.com/v/t51...,"[plant, natural, landscape, terrestrial, plant...",97,1
1,1,1,2,"Photo by @beverlyjoubert | My, what teeth you ...",195558,https://scontent-hou1-1.cdninstagram.com/v/t51...,"[plant, plant, community, felidae, carnivore, ...",93,1
2,2,2,3,Photo by @acacia.johnson | The first rays of a...,121855,https://scontent-hou1-1.cdninstagram.com/v/t51...,"[water, nature, slope, natural, landscape, mou...",117,1
3,3,3,4,Photo by @jasperdoest | A white stork searches...,89130,https://scontent-hou1-1.cdninstagram.com/v/t51...,"[bird, sky, vertebrate, nature, extinction, ne...",80,0
4,4,4,5,Photo by Robbie Shone @shonephoto | A cave sur...,55653,https://scontent-hou1-1.cdninstagram.com/v/t51...,"[flash, photography, outdoor, recreation, recr...",131,0


In [ ]:
df['Label'][0]

'Plant, Natural landscape, Terrestrial plant, Wood, Trunk, Tree, Deciduous, Landscape, Grass, Twig'

In [ ]:
df['Label']=df['Label'].str.replace('-',' ').str.lower()
df['Label']=df['Label'].str.replace(' ','%').str.lower()
df['Label']=df['Label'].str.replace(',','').str.lower()
df['Label']=df['Label'].str.replace('%',', ').str.lower()

df['Label']=df['Label'].apply(lambda x: [''.join(word) for word in x.split(', ')])

In [ ]:
df['Label'][0]

['plant',
 'natural',
 'landscape',
 'terrestrial',
 'plant',
 'wood',
 'trunk',
 'tree',
 'deciduous',
 'landscape',
 'grass',
 'twig']

In [ ]:
convert_to_list = []
for label in df['Label']:
    convert_to_list.extend(label)

convert_to_words=[]
for word in convert_to_list:
    convert_to_words.extend(word.split(' '))

convert_to_words=[word.lower() for word in convert_to_words]

word_dict={}

for word in convert_to_words:
  if word in word_dict.keys():
    word_dict[word]+=1
  else:
    word_dict[word]=1

words_df=pd.DataFrame({'Words':word_dict.keys(), 'Count':word_dict.values()})

In [ ]:
words_df[words_df['Count']>=100]

,Words,Count
0,plant,458
1,natural,323
2,landscape,414
3,terrestrial,275
4,wood,136
6,tree,153
8,grass,108
12,carnivore,107
14,organism,150
19,water,274


In [ ]:
words_df=words_df.sort_values(by=['Count'],ascending=False)
words_df.to_csv('Word_Count.csv')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from gensim import corpora
from gensim.models import LdaModel
import ast
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

custom_stop_words = ['plant', 'landscape', 'natural', 'sky', 'cloud', 'nature', 'tree', 'organism'
, 'wood', 'terrestrial', 'water', 'font', 'photography', 'and', 'in', 'of', 'snout', 'animal'
, 'gesture', 'caption', 'environment', 'community', 'grass', 'people', 'ecoregion'
, 'happy', 'light', 'eye', 'adaption', 'event', 'flash', 'blue', 'design']


# Update the NLTK stop words with your custom words
stop_words.update(custom_stop_words)


convert_to_list = []
for label in df['Label']:
    convert_to_list.append(label)

convert_to_list = [[word.lower() for word in sublist if word.lower() not in custom_stop_words] for sublist in convert_to_list]


# Create a document-term matrix
dictionary = corpora.Dictionary(convert_to_list)
corpus = [dictionary.doc2bow(text) for text in convert_to_list]

In [ ]:
def lda_coherence(number):

  lda = LdaModel(corpus, num_topics=number, id2word=dictionary, passes=15)
  topics = lda.print_topics(num_words=100)

  # Display all words
  topics_data = {}
  for topic_id, topic in lda.show_topics(num_topics=number, num_words=100, formatted=False):
      words = [word for word, score in topic]
      scores = [score for word, score in topic]

      for word, score in zip(words, scores):
          if word in topics_data:
              topics_data[word][f'Topic {topic_id}'] = score
          else:
              topics_data[word] = {f'Topic {topic_id}': score}

  df_topic = pd.DataFrame.from_dict(topics_data, orient='index')

  df_topic.to_csv(f"topic{number}.csv")

# Compute Coherence Score using the C_V measure
  coherence_model_lda = CoherenceModel(model=lda, texts=convert_to_list, dictionary=dictionary, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print(f'Coherence Score (C_V): {number} : ', coherence_lda)

In [ ]:
num_topics = range(8,11)

for number in num_topics:
  lda_coherence(number)

Coherence Score (C_V): 8 :  0.3758815662139832
Coherence Score (C_V): 9 :  0.41039374562994896
Coherence Score (C_V): 10 :  0.3715596491699505


In [ ]:
lda_coherence(8)

Coherence Score (C_V): 8 :  0.4315910480362313


In [ ]:
df_topic=pd.read_csv("topic8.csv")
df_topic=df_topic.rename(columns={'Unnamed: 0':'Label_Words'})
df_topic.fillna(0, inplace = True)

df_topic

,Label_Words,Topic 0,Topic 1,Topic 4,Topic 2,Topic 6,Topic 7,Topic 5,Topic 3
0,carnivore,0.082312,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,fawn,0.058634,0.005348,0.011141,0.0,0.0,0.0,0.0,0.000000
2,felidae,0.045070,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,dog,0.043417,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,whiskers,0.043404,0.002749,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
525,grey,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.002563
526,planet,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.002562
527,outer,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.002562
528,thunder,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.002561


In [ ]:
column_names = df_topic.columns
score_list_list = []

for label in convert_to_list:
  score_list = []
  sum_score=0
  for topic in column_names[1:len(column_names)]:
    score = 0
    for item in label:
      if item in list(df_topic['Label_Words']):
        score += df_topic[df_topic['Label_Words']==item][topic].item()
    score_list.append(score)
    sum_score+=score

  if sum_score != 0:
    results = [number / sum_score for number in score_list]
  else:
    results = [0] * len(score_list)
  score_list_list.append(results)

In [ ]:
score_list_list

[[0.0,
  0.060505276285509745,
  0.8111763541791581,
  0.0,
  0.0,
  0.05930350292525252,
  0.0,
  0.06901486661007968],
 [0.9158545190190859,
  0.009652004198471613,
  0.0,
  0.007047329336088741,
  0.010191931352573753,
  0.057254216093780126,
  0.0,
  0.0],
 [0.010788710102256741,
  0.03303318115058661,
  0.01793255883802083,
  0.5496888302025619,
  0.15695194823790481,
  0.023581832767437395,
  0.012727733797821678,
  0.19529520490341],
 [0.09334805600885272,
  0.08775280669593134,
  0.4439051490517412,
  0.009508070989694408,
  0.025236724103117163,
  0.28138388101661815,
  0.04212383735347404,
  0.0167414747805708],
 [0.0,
  0.036024205443455096,
  0.03599250098685189,
  0.48546374203606013,
  0.02329209630727399,
  0.028615319748225496,
  0.2474968034499701,
  0.14311533202816315],
 [0.07319010426302423,
  0.5693339901845683,
  0.06479435148608911,
  0.0,
  0.0,
  0.026308368861945472,
  0.2663731852043728,
  0.0],
 [0.022633782835112082,
  0.0,
  0.17526904731694587,
  0.638833

In [ ]:
df_topic_score

,Topic 0,Topic 1,Topic 4,Topic 2,Topic 6,Topic 7,Topic 5,Topic 3,like,label
0,0.000000,0.060505,0.811176,0.000000,0.000000,0.059304,0.000000,0.069015,97727,"[plant, natural, landscape, terrestrial, plant..."
1,0.915855,0.009652,0.000000,0.007047,0.010192,0.057254,0.000000,0.000000,195558,"[plant, plant, community, felidae, carnivore, ..."
2,0.010789,0.033033,0.017933,0.549689,0.156952,0.023582,0.012728,0.195295,121855,"[water, nature, slope, natural, landscape, mou..."
3,0.093348,0.087753,0.443905,0.009508,0.025237,0.281384,0.042124,0.016741,89130,"[bird, sky, vertebrate, nature, extinction, ne..."
4,0.000000,0.036024,0.035993,0.485464,0.023292,0.028615,0.247497,0.143115,55653,"[flash, photography, outdoor, recreation, recr..."
...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.000000,0.019096,0.000000,0.000000,0.030667,0.933334,0.016903,30109,"[insect, eye, arthropod, ant, terrestrial, pla..."
996,0.748496,0.018160,0.022139,0.211206,0.000000,0.000000,0.000000,0.000000,157059,"[ecoregion, felidae, sky, carnivore, mountain,..."
997,0.043981,0.113466,0.587354,0.008596,0.043607,0.105585,0.031402,0.066010,77895,"[bird, organism, beak, feather, twig, wing, ta..."
998,0.000000,0.146423,0.000000,0.356598,0.038659,0.228021,0.073431,0.156869,36823,"[atmosphere, sky, water, green, natural, lands..."


In [ ]:
df_topic_score = pd.DataFrame(score_list_list, columns=column_names[1:len(column_names)])

df_topic_score['like'] = df['Like'].to_list()

df_topic_score['label'] = df['Label'].to_list()  # Use the correct column name

df_topic_score.to_csv("final_score_results.csv")


In [ ]:
# Sort the DataFrame by the 'like' column
df_sorted = df_topic_score.sort_values(by='like',ascending = False)

# Calculate the first quartile (25th percentile)~Top25% and third quartile (75th percentile) values
first_quartile_value = df_sorted['like'].quantile(0.75)
third_quartile_value = df_sorted['like'].quantile(0.25)

print("First Quartile:",first_quartile_value)
print("Third Quartile:",third_quartile_value)

First Quartile: 142531.5
Third Quartile: 60313.0


In [ ]:
# Filter the DataFrame to get the first and last quartile rows
first_quartile_rows = df_sorted[df_sorted['like'] >= first_quartile_value]
last_quartile_rows = df_sorted[df_sorted['like'] <= third_quartile_value]

# Display the results
print("First Quartile Rows:")
print(first_quartile_rows)

print("\nLast Quartile Rows:")
print(last_quartile_rows)

142531.5
60313.0
First Quartile Rows:
      Topic 0   Topic 1   Topic 4   Topic 2   Topic 6   Topic 7   Topic 5  \
376  0.018495  0.094868  0.041509  0.013373  0.519909  0.048627  0.036009   
188  0.033592  0.173465  0.076267  0.030459  0.430832  0.099041  0.021654   
17   0.889338  0.022148  0.030472  0.005491  0.007941  0.044610  0.000000   
22   0.000000  0.066515  0.735926  0.047697  0.000000  0.073992  0.000000   
167  0.000000  0.062212  0.071551  0.122110  0.108256  0.035765  0.000000   
..        ...       ...       ...       ...       ...       ...       ...   
882  0.000000  0.000000  0.000000  0.101391  0.207736  0.038451  0.038063   
35   0.000000  0.073783  0.070063  0.000000  0.324423  0.172633  0.209270   
699  0.102051  0.154200  0.344523  0.123590  0.000000  0.034130  0.037465   
843  0.014666  0.090457  0.000000  0.792904  0.000000  0.010801  0.000000   
88   0.000000  0.318015  0.270597  0.000000  0.278126  0.133261  0.000000   

      Topic 3    like                

In [ ]:
topic_columns = [col for col in first_quartile_rows.columns if col.startswith('Topic')]

# Initialize dictionaries to store the high and low engagement averages
high_engagement_data = {}
low_engagement_data = {}

# Calculate the mean for each topic in both high and low engagement DataFrames
for topic in topic_columns:
    high_engagement_data[topic] = first_quartile_rows[topic].mean()
    low_engagement_data[topic] = last_quartile_rows[topic].mean()

# Create DataFrames for high and low averages
high_average_df = pd.DataFrame(high_engagement_data, index=['High Average'])
low_average_df = pd.DataFrame(low_engagement_data, index=['Low Average'])


In [ ]:
high_average_df

,Topic 0,Topic 1,Topic 4,Topic 2,Topic 6,Topic 7,Topic 5,Topic 3
High Average,0.200276,0.107944,0.120953,0.129737,0.112548,0.118558,0.101675,0.108309


In [ ]:
low_average_df

,Topic 0,Topic 1,Topic 4,Topic 2,Topic 6,Topic 7,Topic 5,Topic 3
Low Average,0.046135,0.087657,0.131875,0.203483,0.183307,0.077431,0.129631,0.140481
